In [ ]:
import numpy as np
import os
import scipy.io
from sklearn import linear_model
from sklearn import model_selection

import holoviews as hv;  hv.extension('bokeh', logo=False)

In [ ]:
A   = np.random.randn(100,10)                      # Matrix of possible predictors
x   = np.array([0, 0, 1, 0, 0, 0, -1, 0, 0, 0])    # Two nonzero predictors
b   = A @ x + 2*np.random.randn(100)

xL2 = np.linalg.pinv(A) @ b

In [ ]:
reg    = linear_model.LassoCV(cv=10).fit(A, b)

lasso  = linear_model.Lasso(random_state=0, max_iter=10000)
alphas = np.logspace(-4, -0.5, 30)

tuned_parameters = [{'alpha': alphas}]

clf = model_selection.GridSearchCV(lasso, tuned_parameters, cv=10, refit=False)
clf.fit(A, b)

scores = clf.cv_results_['mean_test_score']
scores_std = clf.cv_results_['std_test_score']

# plot error lines showing +/- std. errors of the scores
std_error = scores_std / np.sqrt(10)

In [ ]:
hv.Spread((alphas, scores,std_error),kdims=["alpha"],vdims=["CV score +-std error", "+ std error"]).opts(logx=True, color="gray", alpha=0.1, width=500 )*\
hv.Curve((alphas, scores)).opts(color='red')*\
hv.HLine(np.max(scores)).opts(line_width=1)

In [ ]:
XL1         = linear_model.Lasso(alpha=clf.best_params_['alpha'])
XL1.fit(A,b)
xL1         = XL1.coef_
xL1DeBiased = np.linalg.pinv(A[:,np.abs(xL1)>0]) @ b